In [1]:
from pyiron_atomistics import Project

In [2]:
pr = Project("submit")

In [3]:
active_cores = 0
number_of_cores = 2

In [ ]:
job_dict = {}
job_next_lst = []
while True:
    continue_check = True
    if len(job_next_lst) == 0:
        df = pr.job_table()
        job_next_lst = df[
            (df["status"] == "submitted") & 
            (df["hamilton"] == "Lammps")
        ].id.values
    for i, job_id in enumerate(job_next_lst):
        if job_id not in job_dict:
            job = pr.load(job_id)
            job.status.created = True 
            job.server.run_mode.thread = True
            job_cores = job.server.cores
            active_cores += job_cores
            job.run()
            job_dict[job_id] = [job_cores, job._process]
        if active_cores == number_of_cores:
            job_next_lst = job_next_lst[i+1:]
            break

    while continue_check:
        finhshed_jobs = []
        for k, [c, p] in job_dict.items():
            if not p.is_alive(): 
                finhshed_jobs.append(k)
                active_cores -= c
                continue_check = False
        for k in finhshed_jobs:
            del job_dict[k]

2021-10-15 01:19:13,750 - pyiron_log - INFO - job: lmp_0 id: 1, status: submitted, run job (modal)
2021-10-15 01:19:14,167 - pyiron_log - INFO - job: lmp_0 id: 1, status: collect, output: LAMMPS (30 Jul 2021)
Reading data file ...
  orthogonal box = (0.0000000 0.0000000 0.0000000) to (40.500000 40.500000 40.500000)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  4000 atoms
  read_data CPU = 0.009 seconds
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 7.65
  ghost atom cutoff = 7.65
  binsize = 3.825, bins = 11 11 11
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d
      bin: standard
Setting up Verlet run ...
  Unit style    : metal
  Current step  : 0
  Time step     : 0.001
Per MPI rank memory allocation (min/avg/max) = 6.317 | 6.317 |